In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm



ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002DA558A2E00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002DA558D5240>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Saarthak\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pytho

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
import bs4
loader = WebBaseLoader(
    web_paths=("https://techcrunch.com/2025/07/25/india-bans-streaming-apps-youve-never-heard-of-but-millions-watch/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("wp-block-paragraph")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://techcrunch.com/2025/07/25/india-bans-streaming-apps-youve-never-heard-of-but-millions-watch/'}, page_content='India has ordered the blocking of 25 streaming services —\xa0many with millions of viewers and even paying subscribers — for allegedly promoting “obscene” content, in one of the South Asian nation’s biggest digital crackdowns yet. The order affects lesser-known, but wildly popular services like Ullu and ALTT that cater to the country’s mass-market appetite for adult and edgy entertainment.This week, the Ministry of Information and Broadcasting issued directives to block access to websites and apps linked to 25 streaming services, citing provisions of the Information Technology Act of 2000 and the IT Rules of 2021, TechCrunch learned.The order came months after the National Commission for Protection of Child Rights and a Parliamentary Standing Committee on Information Technology raised concerns about mature content being streamed on these p

In [8]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002DA05467FD0>, search_kwargs={})

In [9]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [10]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [11]:
response=rag_chain.invoke({"input":"How many streaming services India ordered to block?"})
response

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'input': 'How many streaming services India ordered to block?',
 'context': [Document(id='a7ac60f3-d8d5-4150-892c-6b6214b958b1', metadata={'source': 'https://techcrunch.com/2025/07/25/india-bans-streaming-apps-youve-never-heard-of-but-millions-watch/'}, page_content='India has ordered the blocking of 25 streaming services —\xa0many with millions of viewers and even paying subscribers — for allegedly promoting “obscene” content, in one of the South Asian nation’s biggest digital crackdowns yet. The order affects lesser-known, but wildly popular services like Ullu and ALTT that cater to the country’s mass-market appetite for adult and edgy entertainment.This week, the Ministry of Information and Broadcasting issued directives to block access to websites and apps linked to 25 streaming services, citing provisions of the Information Technology Act of 2000 and the IT Rules of 2021, TechCrunch learned.The order came months after the National Commission for Protection of Child Rights and a P

In [12]:
rag_chain.invoke({"input":"Which are those?"})

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'input': 'Which are those?',
 'context': [Document(id='bf92ff8a-49bc-42e7-a060-a78ea513694f', metadata={'source': 'https://techcrunch.com/2025/07/25/india-bans-streaming-apps-youve-never-heard-of-but-millions-watch/'}, page_content='for intermediaries like Google, Apple, and internet providers to fully block access, as these platforms often spread through alternative channels and use social media platforms like Instagram and YouTube to attract viewers.Topics'),
  Document(id='eabdfc5f-7bad-4c19-bceb-b2c809060dcc', metadata={'source': 'https://techcrunch.com/2025/07/25/india-bans-streaming-apps-youve-never-heard-of-but-millions-watch/'}, page_content='exclusively shared with TechCrunch. The wide gap between in-app purchases and downloads is primarily due to their low subscription costs — significantly lower than Netflix and other global platforms in India.Notably, some of these streaming services were either never listed on traditional app stores or had been removed earlier, and instea

In [13]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [14]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002DA05467FD0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [15]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [16]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [23]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What was Ullu growth?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Was it available on Indian Play Store?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Ullu saw nearly 10% year-over-year growth in worldwide traffic, reaching 1.9 million visits in June, and in India, it recorded 18.9% year-over-year growth, reaching 1.8 million visits.


In [24]:
chat_history

[HumanMessage(content='What was Ullu growth?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the text, Ullu saw nearly 10% year-over-year growth in worldwide traffic, reaching 1.9 million visits in June, while in India, it recorded 18.9% year-over-year growth, reaching 1.8 million visits. Additionally, Ullu Digital, the parent company of Ullu, reported a net profit of ₹212.3 million ($2.5 million) for the financial year 2024.', additional_kwargs={}, response_metadata={})]

In [25]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [28]:
conversational_rag_chain.invoke(
    {"input": "India ordered what?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'India ordered the blocking of 25 streaming services for allegedly promoting "obscene" content.'

In [29]:
conversational_rag_chain.invoke(
    {"input": "Tell me more about the services , some of it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

c:\Users\Saarthak\OneDrive\Desktop\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


"The services that were ordered to be blocked include lesser-known but wildly popular services like Ullu and ALTT, which cater to India's mass-market appetite for adult and edgy entertainment."

In [30]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content="I don't know. The provided context doesn't mention Task Decomposition.", additional_kwargs={}, response_metadata={}), HumanMessage(content='From whom the order came after months?', additional_kwargs={}, response_metadata={}), AIMessage(content='The order came after months from the National Commission for Protection of Child Rights and a Parliamentary Standing Committee on Information Technology.', additional_kwargs={}, response_metadata={}), HumanMessage(content='India ordered what?', additional_kwargs={}, response_metadata={}), AIMessage(content='India ordered the blocking of 25 streaming services for allegedly promoting "obscene" content.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me more about the services , some of it?', additional_kwargs={}, response_metadata={}), AIMessage(content="The se